In [32]:
import ssl
import urllib.request

# SSL 인증서 검증 무시
ssl._create_default_https_context = ssl._create_unverified_context

# CIFAR-10 데이터셋 다운로드
url = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'
filename = 'cifar-10-python.tar.gz'
urllib.request.urlretrieve(url, filename)

# 나머지 코드는 데이터셋을 로드하고 모양을 출력하는 코드와 동일합니다.


from tensorflow.keras.datasets import cifar10

(train_images, train_targets), (test_images, test_targets) = cifar10.load_data()

# 타겟 데이터의 dimension은 1로 맞춰준다. # squeeze()
# one-hot encoding의 적용 여부를 판단하기 편하다.
print(train_images.shape, train_targets.squeeze().shape)
print(test_images.shape, test_targets.squeeze().shape)

(50000, 32, 32, 3) (50000,)
(10000, 32, 32, 3) (10000,)


In [33]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np

def get_preprocessed_data(images, targets):
    images = np.array(images / 255.0, dtype=np.float32)
    targets = np.array(targets, dtype=np.float32)

    return images, targets

def get_preprocessed_ohe(images, targets):
    images, targets = get_preprocessed_data(images, targets)
    oh_targets = to_categorical(targets)

    return images, oh_targets

def get_train_valid_test(train_images, train_targets, test_images, test_targets, validation_size=0.2, random_state=124):
    train_images, train_oh_targets = get_preprocessed_ohe(train_images, train_targets)
    test_images, test_oh_targets = get_preprocessed_ohe(test_images, test_targets)

    train_images, validation_images, train_oh_targets, validation_oh_targets = \
    train_test_split(train_images, train_oh_targets, stratify=train_oh_targets, test_size=validation_size, random_state=random_state)

    return (train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets)

In [34]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import  Input, Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

IMAGE_SIZE = 32

input_tensor = Input(shape=(IMAGE_SIZE,IMAGE_SIZE,3))

x = Conv2D(filters=32, kernel_size=5, padding='valid', activation='relu')(input_tensor)
x = Conv2D(filters=32, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPooling2D(2)(x)

x = Conv2D(filters=64, kernel_size=3, padding='valid', activation='relu')(x)
x = Conv2D(filters=64, kernel_size=3, padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling2D(2)(x)

x = Conv2D(filters=128, kernel_size=3, padding='valid', activation='relu')(x)
x = Conv2D(filters=128, kernel_size=3, padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling2D(2)(x)

x = Flatten(name='classifierA00')(x)
x = Dropout(name='classifierA_Dropout01', rate=0.5)(x)
x = Dense(300, activation='relu',name='classifierAD01')(x)
x = Dropout(name='classifierA_Dropout02', rate=0.3)(x)
output = Dense(10, activation='softmax',name='classifierAD02')(x)

model = Model(inputs = input_tensor, outputs = output)
model.summary()

Model: "functional_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)          │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_62 (Conv2D)                   │ (None, 28, 28, 32)          │           2,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_63 (Conv2D)                   │ (None, 28, 28, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_30 (MaxPooling2D)      │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_64 (Conv2D)                   │ (None, 12, 12, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_65 (Conv2D)                   │ (None, 12, 12, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_19 (Activation)           │ (None, 12, 12, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_31 (MaxPooling2D)      │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_66 (Conv2D)                   │ (None, 4, 4, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_67 (Conv2D)                   │ (None, 4, 4, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_20 (Activation)           │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_32 (MaxPooling2D)      │ (None, 2, 2, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ classifierA00 (Flatten)              │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ classifierA_Dropout01 (Dropout)      │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ classifierAD01 (Dense)               │ (None, 300)                 │         153,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ classifierA_Dropout02 (Dropout)      │ (None, 300)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ classifierAD02 (Dense)               │ (None, 10)                  │           3,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 445,454 (1.70 MB)

 Trainable params: 445,454 (1.70 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(), metrics=['acc'])

In [36]:
history = model.fit(x=train_images, y=train_targets, batch_size=128, epochs=30, validation_split=0.2)

Epoch 1/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 17s 44ms/step - acc: 0.2177 - loss: 2.5216 - val_acc: 0.4264 - val_loss: 1.5412
Epoch 2/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 46ms/step - acc: 0.4312 - loss: 1.5371 - val_acc: 0.5275 - val_loss: 1.3266
Epoch 3/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step - acc: 0.5079 - loss: 1.3532 - val_acc: 0.5809 - val_loss: 1.1926
Epoch 4/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step - acc: 0.5619 - loss: 1.2164 - val_acc: 0.5776 - val_loss: 1.1791
Epoch 5/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step - acc: 0.6000 - loss: 1.1238 - val_acc: 0.6067 - val_loss: 1.1146
Epoch 6/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step - acc: 0.6347 - loss: 1.0407 - val_acc: 0.6595 - val_loss: 0.9780
Epoch 7/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 16s 50ms/step - acc: 0.6565 - loss: 0.9572 - val_acc: 0.6588 - val_loss: 0.9713
Epoch 8/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step - acc: 0.6764 - loss: 0.9088 - val_acc: 0.6579 - val_loss: 0.9879
Epoch 9/30
313/313 ━━━━━━━━━━━━━━━━━━━━ 

In [37]:
from tensorflow.keras.datasets import cifar10

(train_images, train_targets), (test_images, test_targets) = cifar10.load_data()

# 타겟 데이터의 dimension은 1로 맞춰준다. # squeeze()
# one-hot encoding의 적용 여부를 판단하기 편하다.
print(train_images.shape, train_targets.squeeze().shape)
print(test_images.shape, test_targets.squeeze().shape)

(50000, 32, 32, 3) (50000,)
(10000, 32, 32, 3) (10000,)


In [38]:
import numpy as np 

def get_preprocessed_data(images, targets):
    images = np.array(images / 255.0, dtype = np.float32)
    targets = np.array(targets, dtype = np.float32)

    return images, targets


train_images, train_targets = get_preprocessed_data(train_images, train_targets)
test_images, test_targets = get_preprocessed_data(test_images, test_targets)

In [39]:
train_targets = train_targets.squeeze()
test_targets = test_targets.squeeze()

print(train_images.shape, train_targets.shape)
print(test_images.shape, test_targets.shape)

(50000, 32, 32, 3) (50000,)
(10000, 32, 32, 3) (10000,)


In [40]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D, Input, Activation
from tensorflow.keras.callbacks import Callback

INPUT_SIZE = 32

input_tensor = Input(shape=(INPUT_SIZE, INPUT_SIZE, 3))

# padding default == valid
x = Conv2D(filters = 32, kernel_size=5, padding='valid', activation='relu')(input_tensor)
x = Conv2D(filters = 32, kernel_size=3, padding='same', activation='relu')(x)
x = MaxPooling2D(2)(x)

x = Conv2D(filters = 64, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D(filters = 64, kernel_size=3, padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling2D(2)(x)

x = Conv2D(filters = 128, kernel_size=3, padding='same', activation='relu')(x)
x = Conv2D(filters = 128, kernel_size=3, padding='same')(x)
x = Activation('relu')(x)
x = MaxPooling2D(2)(x)

x = Flatten(name='classifier_A00_Flatten')(x)
x = Dropout(name='classifierA_DropOut01', rate=0.5)(x)
x = Dense(300, activation='relu', name='classifierAD01')(x)
x = Dropout(name='classifierA_DropOut02', rate=0.5)(x)
output = Dense(10, activation='softmax', name='output')(x)


model = Model(inputs = input_tensor, outputs = output)
model.summary()

Model: "functional_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)          │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_68 (Conv2D)                   │ (None, 28, 28, 32)          │           2,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_69 (Conv2D)                   │ (None, 28, 28, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_33 (MaxPooling2D)      │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_70 (Conv2D)                   │ (None, 14, 14, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_71 (Conv2D)                   │ (None, 14, 14, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_21 (Activation)           │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_34 (MaxPooling2D)      │ (None, 7, 7, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_72 (Conv2D)                   │ (None, 7, 7, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_73 (Conv2D)                   │ (None, 7, 7, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_22 (Activation)           │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_35 (MaxPooling2D)      │ (None, 3, 3, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ classifier_A00_Flatten (Flatten)     │ (None, 1152)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ classifierA_DropOut01 (Dropout)      │ (None, 1152)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ classifierAD01 (Dense)               │ (None, 300)                 │         345,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ classifierA_DropOut02 (Dropout)      │ (None, 300)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 10)                  │           3,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 637,454 (2.43 MB)

 Trainable params: 637,454 (2.43 MB)

 Non-trainable params: 0 (0.00 B)

In [41]:
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.losses import CategoricalCrossentropy
# 내가 원=핫 인코딩을 하지않고 함수 내부적으로 원-핫 인코딩을 시켜준다.
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(), metrics = ['acc'])

In [42]:
history = model.fit(x=train_images,
                    y=train_targets, 
                    batch_size=128,
                    epochs=20,
                    validation_split=0.2)

Epoch 1/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 24s 69ms/step - acc: 0.2050 - loss: 2.0698 - val_acc: 0.4244 - val_loss: 1.5783
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 70ms/step - acc: 0.4253 - loss: 1.5460 - val_acc: 0.5272 - val_loss: 1.2967
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 70ms/step - acc: 0.5220 - loss: 1.3216 - val_acc: 0.5846 - val_loss: 1.1662
Epoch 4/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 23s 74ms/step - acc: 0.5827 - loss: 1.1609 - val_acc: 0.6414 - val_loss: 1.0248
Epoch 5/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 70ms/step - acc: 0.6302 - loss: 1.0450 - val_acc: 0.6543 - val_loss: 0.9829
Epoch 6/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 70ms/step - acc: 0.6649 - loss: 0.9425 - val_acc: 0.6932 - val_loss: 0.8701
Epoch 7/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 71ms/step - acc: 0.7010 - loss: 0.8465 - val_acc: 0.7103 - val_loss: 0.8301
Epoch 8/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 22s 71ms/step - acc: 0.7208 - loss: 0.7884 - val_acc: 0.7135 - val_loss: 0.8125
Epoch 9/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 

In [43]:
model.evaluate(test_images, test_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - acc: 0.7787 - loss: 0.7311


[0.7437151670455933, 0.7742000222206116]

In [46]:
pred_probas = model.predict(test_images[8:16], batch_size=128)
pred_classes = np.argmax(pred_probas.squeeze(), axis=1)

(test_images[8:16], pred_classes)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


NameError: name 'show_images' is not defined